In [1]:
import numpy as np
import pandas as pd
import psycopg2

### Connect to Postgres DB

In [2]:
import json

with open('config.json') as f:
    conf = json.load(f)
    host = conf['host']
    database = conf['database']
    user = conf['user']
    passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)
conn = psycopg2.connect(conn_str)

### Initial Query

In [ ]:
Vic_Off_query = 
'''
SELECT  vic.victim_id AS VICTIM_ID,
        vic.incident_id AS INCIDENT_ID,
        date_part('year',inc.incident_date),
        vic.victim_type_id AS VICTIM_TYPE_ID,
        ty.victim_type_name AS VICTIM_TYPE,
        vic.age_range_low_num AS AGE_RANGE_LOW,
        vic.age_range_high_num AS AGE_RANGE_HIGH,
        vic.sex_code AS VICTIM_SEX,
        oft.crime_against AS CRIME_AGAINST,
        oft.offense_name AS OFFENSE,
        oft.offense_category_name AS OFFENSE_CATEGORY,
        oft.offense_group AS OFFENSE_GROUP,
        ofr.offender_id AS OFFENDER_ID,
        ofr.age_num AS OFFENDER_AGE,
        ofr.sex_code AS OFFENDER_SEX,
        off.location_id AS LOCATION_ID,
        loc.location_name AS LOCATION_NAME,
        inc.ddocname AS DOC_NAME,
        ori.fips AS FIPS,
        ori.countyname AS COUNTY,
        ori.name AS ORI_NAME,
        ags.population AS POPULATION,
        ags.population_group_desc AS POPULATION DESCRIPTION,
        ags.total_officers AS OFFICERS,
        ags.total_civilians AS CIVILIANS
        
FROM    nibrs_victim as vic
JOIN    nibrs_victim_type as ty
ON      vic.victim_type_id = ty.victim_type_id
JOIN    nibrs_offender as ofr
ON      ofr.incident_id = vic.incident_id
JOIN    nibrs_offense as off
ON      off.incident_id = vic.incident_id
JOIN    nibrs_offense_type as oft
ON      oft.offense_type_id = off.offense_type_id
JOIN    nibrs_location_type as loc
ON      off.location_id = loc.location_id
JOIN    nibrs_incident as inc
ON      inc.incident_id = vic.incident_id
JOIN    cde_agencies as ags
ON      ags.agency_id = inc.agency_id
JOIN    ori_to_fips as ori
ON      ori.ori9 = ags.ori

WHERE   vic.victim_type_id = 4;
'''

### Load into Pandas DataFrame

In [4]:
Vic_Off_df = pd.read_csv('init_query.csv')
Vic_Off_df.head()

,victim_id,incident_id,victim_type_id,victim_type,age_range_low,age_range_high,victim_sex,crime_against,offense,offense_category,offense_group,offender_id,offender_age,offender_sex,location_id,location_name,ddocname,fips,countyname,name
0,57023969,52643345,4,Individual,NaN,NaN,M,Property,Theft of Motor Vehicle Parts or Accessories,Larceny/Theft Offenses,A,59269093,NaN,U,18,Parking/Drop Lot/Garage,2010_07_TX2201200_100074401_INC_NIBRS,48439,TARRANT,FORT WORTH POLICE DEPARTMENT
1,57023969,52643345,4,Individual,NaN,NaN,M,Property,Theft of Motor Vehicle Parts or Accessories,Larceny/Theft Offenses,A,59269093,NaN,U,18,Parking/Drop Lot/Garage,2010_07_TX2201200_100074401_INC_NIBRS,48439,TARRANT,FORT WORTH POLICE DEPARTMENT
2,57023969,52643345,4,Individual,NaN,NaN,M,Property,Theft of Motor Vehicle Parts or Accessories,Larceny/Theft Offenses,A,59269093,NaN,U,18,Parking/Drop Lot/Garage,2010_07_TX2201200_100074401_INC_NIBRS,48439,TARRANT,FORT WORTH POLICE DEPARTMENT
3,57023969,52643345,4,Individual,NaN,NaN,M,Property,Theft of Motor Vehicle Parts or Accessories,Larceny/Theft Offenses,A,59269093,NaN,U,18,Parking/Drop Lot/Garage,2010_07_TX2201200_100074401_INC_NIBRS,48439,TARRANT,FORT WORTH POLICE DEPARTMENT
4,57023969,52643345,4,Individual,NaN,NaN,M,Property,Theft of Motor Vehicle Parts or Accessories,Larceny/Theft Offenses,A,59269093,NaN,U,18,Parking/Drop Lot/Garage,2010_07_TX2201200_100074401_INC_NIBRS,48439,TARRANT,FORT WORTH POLICE DEPARTMENT


### Count of NaN's by column

In [5]:
Vic_Off_df.isna().sum()

victim_id                 0
incident_id               0
victim_type_id            0
victim_type               0
age_range_low       8332860
age_range_high      8332860
victim_sex                0
crime_against             0
offense                   0
offense_category          0
offense_group             0
offender_id               0
offender_age        4539013
offender_sex        2291595
location_id               0
location_name             0
ddocname                  0
fips                      0
countyname                0
name                      0
dtype: int64

### Impute missing offender_sex as "U" (unknown)

In [6]:
Vic_Off_df.offender_sex.value_counts()

M    4290919
U    1727647
F    1560047
Name: offender_sex, dtype: int64

In [64]:
Vic_Off_df['offender_sex'].fillna('U', inplace=True)

In [65]:
Vic_Off_df.offender_sex.value_counts()

M    4290919
U    4019242
F    1560047
Name: offender_sex, dtype: int64

### Look at agency participation

In [16]:
agency_part_query = "SELECT * FROM agency_participation;"

agency_part_df = pd.read_sql(agency_part_query, con=conn)

In [17]:
agency_df.year.value_counts()

2016    2330
2015    2296
2014    2282
2013    1134
2012    1131
2011    1121
2010    1113
Name: year, dtype: int64

In [18]:
agency_query = "SELECT * FROM cde_agencies;"

agency_df = pd.read_sql(agency_query, con=conn)

In [31]:
agency_df.head(3).T

,0,1,2
agency_id,19027,19313,19409
ori,TX0572100,TX1080500,TX1290000
legacy_ori,TX0572100,TX1080500,TX1290000
agency_name,Sachse Police Department,Elsa Police Department,Kaufman County Sheriff's Office
short_name,Sachse,Elsa,Kaufman
agency_type_id,1,1,2
agency_type_name,City,City,County
tribe_id,NaN,NaN,NaN
campus_id,NaN,NaN,NaN
city_id,10730,10392,NaN


In [28]:
agency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6940 entries, 0 to 6939
Data columns (total 44 columns):
agency_id                   6940 non-null int64
ori                         6940 non-null object
legacy_ori                  6940 non-null object
agency_name                 6940 non-null object
short_name                  6940 non-null object
agency_type_id              6940 non-null int64
agency_type_name            6940 non-null object
tribe_id                    7 non-null float64
campus_id                   485 non-null float64
city_id                     4568 non-null float64
city_name                   4565 non-null object
state_id                    6940 non-null int64
state_abbr                  6940 non-null object
primary_county_id           6940 non-null int64
primary_county              6927 non-null object
primary_county_fips         6927 non-null object
agency_status               6940 non-null object
submitting_agency_id        6940 non-null int64
submitting_sai   

In [30]:
agency_df.head(3).T

,0,1,2
agency_id,19027,19313,19409
ori,TX0572100,TX1080500,TX1290000
legacy_ori,TX0572100,TX1080500,TX1290000
agency_name,Sachse Police Department,Elsa Police Department,Kaufman County Sheriff's Office
short_name,Sachse,Elsa,Kaufman
agency_type_id,1,1,2
agency_type_name,City,City,County
tribe_id,NaN,NaN,NaN
campus_id,NaN,NaN,NaN
city_id,10730,10392,NaN


In [34]:
df_2016 = pd.read_csv('init_2016.csv')
df_2016.head()

,victim_id,incident_id,year,victim_type_id,victim_type,age_range_low,age_range_high,victim_sex,crime_against,offense,...,offense_group,offender_id,offender_age,offender_sex,location_id,location_name,doc_name,fips,county,ori_name
0,95384706,87276968,2016,4,Individual,25.0,0.0,M,Property,Pocket-picking,...,A,98477248,NaN,NaN,8,Department/Discount Store,2016_01_TX0140700_16000156_INC,48027,BELL,TEMPLE POLICE DEPARTMENT
1,95384706,87276968,2016,4,Individual,25.0,0.0,M,Property,Pocket-picking,...,A,98477248,NaN,NaN,8,Department/Discount Store,2016_01_TX0140700_16000156_INC,48027,BELL,TEMPLE POLICE DEPARTMENT
2,95384706,87276968,2016,4,Individual,25.0,0.0,M,Property,Pocket-picking,...,A,98477248,NaN,NaN,8,Department/Discount Store,2016_01_TX0140700_16000156_INC,48027,BELL,TEMPLE POLICE DEPARTMENT
3,95384706,87276968,2016,4,Individual,25.0,0.0,M,Property,Pocket-picking,...,A,98477248,NaN,NaN,8,Department/Discount Store,2016_01_TX0140700_16000156_INC,48027,BELL,TEMPLE POLICE DEPARTMENT
4,95384706,87276968,2016,4,Individual,25.0,0.0,M,Property,Pocket-picking,...,A,98477248,NaN,NaN,8,Department/Discount Store,2016_01_TX0140700_16000156_INC,48027,BELL,TEMPLE POLICE DEPARTMENT


In [35]:
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353515 entries, 0 to 1353514
Data columns (total 21 columns):
victim_id           1353515 non-null int64
incident_id         1353515 non-null int64
year                1353515 non-null int64
victim_type_id      1353515 non-null int64
victim_type         1353515 non-null object
age_range_low       1321603 non-null float64
age_range_high      1321603 non-null float64
victim_sex          1353515 non-null object
crime_against       1353515 non-null object
offense             1353515 non-null object
offense_category    1353515 non-null object
offense_group       1353515 non-null object
offender_id         1353515 non-null int64
offender_age        731612 non-null float64
offender_sex        1028947 non-null object
location_id         1353515 non-null int64
location_name       1353515 non-null object
doc_name            1353515 non-null object
fips                1353515 non-null int64
county              1353515 non-null object
ori_name    

In [36]:
df_2016.corr()

,victim_id,incident_id,year,victim_type_id,age_range_low,age_range_high,offender_id,offender_age,location_id,fips
victim_id,1.000000,0.999995,NaN,NaN,0.002099,0.003555,0.999996,0.016092,-0.000448,-0.021333
incident_id,0.999995,1.000000,NaN,NaN,0.002131,0.003562,0.999999,0.016140,-0.000446,-0.021048
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
victim_type_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age_range_low,0.002099,0.002131,NaN,NaN,1.000000,-0.036722,0.002140,0.284103,-0.033257,-0.015095
age_range_high,0.003555,0.003562,NaN,NaN,-0.036722,1.000000,0.003574,-0.004259,-0.003072,-0.011174
offender_id,0.999996,0.999999,NaN,NaN,0.002140,0.003574,1.000000,0.016132,-0.000453,-0.021308
offender_age,0.016092,0.016140,NaN,NaN,0.284103,-0.004259,0.016132,1.000000,-0.055057,0.014825
location_id,-0.000448,-0.000446,NaN,NaN,-0.033257,-0.003072,-0.000453,-0.055057,1.000000,-0.022081
fips,-0.021333,-0.021048,NaN,NaN,-0.015095,-0.011174,-0.021308,0.014825,-0.022081,1.000000


In [37]:
df_2016.isna().sum()

victim_id                0
incident_id              0
year                     0
victim_type_id           0
victim_type              0
age_range_low        31912
age_range_high       31912
victim_sex               0
crime_against            0
offense                  0
offense_category         0
offense_group            0
offender_id              0
offender_age        621903
offender_sex        324568
location_id              0
location_name            0
doc_name                 0
fips                     0
county                   0
ori_name                 0
dtype: int64

In [38]:
df_2016['offender_sex'].fillna('U', inplace=True)

In [39]:
df_2016.isna().sum()

victim_id                0
incident_id              0
year                     0
victim_type_id           0
victim_type              0
age_range_low        31912
age_range_high       31912
victim_sex               0
crime_against            0
offense                  0
offense_category         0
offense_group            0
offender_id              0
offender_age        621903
offender_sex             0
location_id              0
location_name            0
doc_name                 0
fips                     0
county                   0
ori_name                 0
dtype: int64